In [1]:
import numpy as np
import cv2

In [2]:
my_faces = np.load('my_faces.npy')
unknown_faces = np.load('unknown_faces.npy')

In [3]:
import matplotlib.pyplot as plt

In [4]:
#plt.imshow(unknown_faces[0])

In [5]:
data = np.r_[my_faces.reshape(400,-1), unknown_faces.reshape(400,-1)]

In [6]:
data = data / 255.0
data.shape

(800, 7500)

In [7]:
data.shape

(800, 7500)

In [8]:
labels = np.zeros((data.shape[0],1))

In [9]:
labels[400:,:] = 1.0

In [10]:
labels[:10]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [11]:
np.unique(labels,return_counts=True)

(array([0., 1.]), array([400, 400], dtype=int64))

In [12]:
names = {
    0 : "KASHISH" ,
    1 : "UNKNOWN"
}

In [13]:
np.random.seed(101)
np.random.shuffle(data)
np.random.seed(101)
np.random.shuffle(labels)

In [14]:
data = np.c_[data, labels]

In [15]:
data

array([[0.54901961, 0.52156863, 0.5372549 , ..., 0.69411765, 0.78039216,
        0.        ],
       [0.98039216, 0.25490196, 0.91372549, ..., 0.25490196, 0.91372549,
        1.        ],
       [0.55686275, 0.45490196, 0.43529412, ..., 0.2627451 , 0.15686275,
        0.        ],
       ...,
       [0.98039216, 0.25490196, 0.91372549, ..., 0.25490196, 0.91372549,
        1.        ],
       [0.45490196, 0.4       , 0.42352941, ..., 0.22352941, 0.1254902 ,
        0.        ],
       [0.98039216, 0.25490196, 0.91372549, ..., 0.25490196, 0.91372549,
        1.        ]])

In [16]:
np.random.shuffle(data)

In [17]:
labels = data[:,-1]

In [18]:
labels

array([1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
       1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0.,
       1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1.,
       0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0.,
       0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1.,
       1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1.,
       0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
       0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1.,
       0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1.,
       0., 1., 1., 1., 1.

In [19]:
data = data[:,0:-1]

In [20]:
data.shape

(800, 7500)

In [21]:
#plt.imshow(data[0].reshape(50,50))

In [22]:
def distance(x1,x2):
    return np.sqrt(np.sum((x1-x2) ** 2))
    

In [25]:
def knn(data, target, k=11):
    dist = []
    for i in range(len(data)):
        dist.append(distance(data[i], target))
    #print(dist)
    sorted_index = np.argsort(dist)
    sorted_labels = labels[sorted_index][:k]
    #print(sorted_labels)
    unique_labels = np.unique(sorted_labels, return_counts=True)
    #print(unique_labels)
    majority_element = np.argmax(unique_labels[1])
    return majority_element

In [32]:
haardata = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('video_1.wmv')
faceData = []
font = cv2.FONT_HERSHEY_COMPLEX
while True:
    flag, img = cap.read()
    if flag:
        img = cv2.resize(img, None, fx=0.5, fy=0.5)
        faces = haardata.detectMultiScale(img,1.2)
        for x,y,w,h in faces:
            cv2.rectangle(img, (x,y), (x+w, y+h), (0,0,255), 5)
            face = img[y:y+h, x:x+w, :]
            face = cv2.resize(face, (50,50))
            #face = face / 255.0
            pred = knn(data, face.reshape(1,-1))
            name = names[int(pred)]
            #print(pred)
            cv2.putText(img, name, (x,y), font, 1, (0,255,0),1)
        cv2.imshow('img', img)
        if cv2.waitKey(1) == 27 or len(faceData) >= 400:
            break
    else:
        print("Camera is not detected...")
        break
cv2.destroyAllWindows()
cap.release()